# 4. Regresión logística y Naive Bayes- Detección de noticias falsas

José Luis Aguilera Luzania

## Introducción

**¿Qué es la detección de *Fake News*?**
La detección de noticias falsas (Fake News) es la tarea de evaluar la veracidad de las afirmaciones en las noticias. Este es un problema crítico en el Procesamiento del Lenguaje Natural (PLN) porque tanto en medios de noticias tradicionales como en medios digitales las Fake News generan un gran impacto social y político en cada individuo. Por ejemplo, la exposición a las Fake News puede generar actitudes de ineficacia, alienación y cinismo hacia ciertos candidatos políticos (Balmas, 2014).

**Objetivo de la libreta**
El objetivo de esta libreta clasificar las noticias utilizando los métodos de Regresión Logística y Naive Bayes Multinomial.

## Librerías y datos

**Librerías**
- Manipulación de datos:
    - `pandas`: Librería para manipular los datos de forma tabular.
    - `matplotlib`: Librería para graficar.
    - `cmd`: Librería para controlar el formato de impresión en la consola.
    - `re`: Librería para utilizar expresiones regulares.

- Procesamiento del lenguaje natural:
    - `nltk`: Librería para utilizar técnicas de procesamiento del lenguaje natural.

- Representación de los datos:
    - `wordcloud`: Librería para generar una nube de palabras y guardarla como imagen *.png*.

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cmd
import nltk
import unidecode

from nltk.corpus import stopwords
from wordcloud import WordCloud

import re
import pickle
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### Datos adicionales para la librería `nltk`
- `punkt`: Necesario para utilizar el tokenizador de los textos.
- `stopwords`: Palabras comunes que no añaden información, como: el, la, los, etc.

In [35]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JoseLuis_AL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JoseLuis_AL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Leer los datos

Los datos están divididos en los archivos `train.xlsx` y `development.xlsx`, con 80% para entrenamiento y 20% para pruebas respectivamente.
Para cargar los conjuntos de datos se utilizará la librería `pandas` y su estructura `DataFrame`.

In [36]:
df_datos = pd.read_csv('Datos/datos.csv')
df_datos.head()

,Categoria,Tema,Fuente,Encabezado,Texto
0,falsa,educación,el ruinaversal,rae incluira la palabra lady en el diccionario...,rae incluira la palabra lady en el diccionario...
1,falsa,educación,hay noticia,la palabra haiga aceptada por la rae,la palabra haiga aceptada por la rae la real a...
2,falsa,educación,el ruinaversal,yordi rosado escribira y disenara los nuevos l...,yordi rosado escribira y disenara los nuevos l...
3,verdadera,educación,el universal,unam capacitara a maestros para aprobar prueba...,unam capacitara a maestros para aprobar prueba...
4,falsa,educación,lamula,pretenden aprobar libros escolares con conteni...,alerta pretenden aprobar libros escolares con ...


## Regresión logística

La regresión logística es un modelo matemático empleado para predecir qué tan probable es que ocurra un evento teniendo en cuenta datos previos. Esta funciona con datos binarios, es decir, cuando un evento ocurre o no.

![Regresión logística](./imagenes/regresion_logistica.png)

Como se observa en la gráfica, $S-Curve$ representa una relación no lineal entre $X$ y $Y$. La letra $S$ significa sigmoide, ya que la función logística es una función sigmoide, la cual se representa con la siguiente ecuación:

$$
Y = \frac{1}{1+e^{-Z}} \\
Z = w.X+b
$$

### Separar el texto de las categorías.

In [37]:
X = df_datos['Texto']
Y = df_datos['Categoria']

In [38]:
Y = Y.astype('category').cat.codes

In [39]:
# Obtener las stopwords.
stopwords_spanish_nltk = stopwords.words('spanish')
stopwords_spanish_json = list(pd.read_json('stopwords-es.json')[0])

# Se concatenan las stopwords, se crea un set para eliminar repetidos y sé genera una lista.
stopwords_spanish = list(set(stopwords_spanish_nltk+stopwords_spanish_json))

# Se agrega la stopword 'NUMBER', debido a que no aporta nada a los datos para el análisis.
stopwords_spanish.append('number')

In [40]:
# Stemmer para palabras en español.
snowball_stem = SnowballStemmer('spanish')

def stemming(texto):

    # Texto a minúsculas.
    texto_stem = texto.lower()

    # Eliminar caracteres especiales.
    texto_stem = unidecode.unidecode(texto)

    # Agrupar solo las palabras.
    texto_stem = re.findall(r'\w+', texto_stem)
    texto_stem = ' '.join(texto_stem)

    # Separar las palabras.
    texto_stem = texto_stem.split()

    # Stemming.
    texto_stem = [snowball_stem.stem(palabra) for palabra in texto_stem if not palabra in stopwords_spanish]

    # Agrupar los tokens resultantes.
    texto_stem = ' '.join(texto_stem)

    return texto_stem

In [41]:
X = X.apply(stemming)

In [42]:
X = X.values
Y = Y.values

In [43]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [44]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, stratify=Y, random_state=2021)

model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [45]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
training_data_accuracy

0.9957716701902748

In [46]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
test_data_accuracy

0.7142857142857143

In [47]:
X_new = X_test[3]
prediction = model.predict(X_new)
print(prediction)
if (prediction[0]==1):
  print('La noticia es probablemente verdadera')
else:
  print('La noticia es probablemente falsa')

[1]
La noticia es probablemente verdadera


In [48]:
def detect_news(content):
    ## Aplicamos Stemming al texto
    stemmed_data = [stemming(content)]
    ## Lo convertimos en datos numéricos
    vectorized_data = vectorizer.transform(stemmed_data)
    ## Realizamos la predicción
    prediction = model.predict(vectorized_data)
    if (prediction[0]==1):
      print('La noticia es probablemente verdadera')
    else:
      print('La noticia es probablemente falsa')

In [49]:
# Verdadera
detect_news('El Gobierno de la Ciudad de México anunció el reforzamiento del programa ‘Conduce sin Alcohol’, con el cual se instalarán puntos de alcoholímetro en plazas comerciales, restaurantes y bares, así como en espacios públicos. Por órdenes de la jefa de Gobierno, Claudia Sheinbaum, se intensificará la presencia y operación del programa en las 16 alcaldías, incluyendo las entradas y salidas de la ciudad, donde se contará con la participación del personal de la Secretaría de Movilidad (Semovi). A partir de este 7 de diciembre y hasta el 16 de enero, habrá 23 puntos de prueba diarios: siete puntos para las jornadas diurnas, dos de ellos en zonas carreteras; y 16 para las jornadas nocturnas, con 10 puntos fijos y 6 itinerarios. En conferencia de prensa, el secretario de seguridad ciudadana de la CDMX, Omar García Harfuch, informó que se instalará alcoholímetro en plazas comerciales, en eventos públicos, durante eventos deportivos, en las explanadas de las alcaldías, así como en las zonas de restaurantes y bares. En esos lugares se instalarán dos puntos diarios de “pruebas amistosas”, en horarios de 12:00 a 18:00 horas y de 19:00 a 23:00. A su vez, la Secretaría de Seguridad Ciudadana fortalecerá los operativos viales en Anillo Periférico, Circuito Bicentenario y Calzada General Ignacio Zaragoza. García Harfuch señaló que para la implementación del operativo cuentan con herramientas tecnológicas nuevas, como lo son 94 videocámaras corporales y 30 equipos de alcostop, un medidor que permite detectar niveles de alcohol a distancia. El objetivo del reforzamiento de los puntos de alcoholímetro, indicó el secretario, es “salvaguardar la integridad física de conductores de vehículos, peatones, ciclistas y la ciudadanía en general, así como prevenir accidentes o hechos de tránsito ante el consumo inmoderado de alcohol”. También se realizarán “jornadas amistosas” con pláticas para prevenir accidentes relacionados con el consumo de alcohol. Estas se llevarán a cabo en plazas comerciales, parques públicos, explanadas de alcaldías, eventos deportivos, así como en restaurantes y bares.')

La noticia es probablemente verdadera


In [50]:
# Falsa
detect_news('Ahora Ricardo Anaya pretende poner un Starbucks en cada municipio Los Whitexicans.- Apenas a unas semanas de que Ricardo Anaya anunciara su gira por mil municipios del país para “vivir los problemas como propios”, el excandidato ya se rindió. De acuerdo con fuentes cercanas al ¿político?, Anaya la estuvo pasando mal en los pueblos, pero la gota que derramó el vaso fue que no encontrara ningún Starbucks abierto. Ricardo Anaya estuvo de gira por comunidades rurales para ganarse la simpatía de la gente con miras a la elección presidencial de 2024. Y aunque parecía que al principio todo sería como un viaje normal de cualquier whitexican a un pueblo mágico, poco a poco se fue dando cuenta de la realidad de su situación. A la semana de iniciar el recorrido se le fue terminando el jamón ibérico, las latas de paté importado y todas sus otras provisiones finas. Sin embargo, el verdadero problema surgió cuando se terminó las botellas de café de Starbucks que había llevado consigo. Desesperado, Ricardo Anaya comenzó a preguntar en los pueblos por el Starbucks más cercano sin éxito. No aceptó un cafecito de olla Los pobladores le ofrecieron café de olla con piloncillo y canela, pero al estar acostumbrado a que su café lleve crema, chispas de chocolate y leche deslactosada, se vio en la necesidad de rechazarlo. Finalmente, desistió de su gira y renovó sus compromisos de precampaña. Ahora leerá los artículos de Wikipedia de mil municipios del país desde la comodidad de su oficina en Atlanta, que queda junto a un Starbucks.')

La noticia es probablemente falsa
